In [2]:
# pip install transformers datasets optimum[onnxruntime] accelerate torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

# 加载模型 
# 模型介绍：distilbert-base-uncased-finetuned-sst-2-english 是一个经过微调的 DistilBERT 模型，用于情感分析任务，能够将文本分类为正面或负面情绪。
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# print("模型加载完成")
# print(model)

# 量化校准数据集准备

由于该模型使用于情感分类的，所以我们选择一个情感分析的数据集中的小部分数据，用于在推理期间观察激活值的分布，从而进行量化校准。

In [3]:
from datasets import load_dataset
# 加载数据集 
dataset_name = "sst2" # 斯坦福的情感分类数据集 是个二分类数据集
dataset = load_dataset(dataset_name, split="train[:5%]")  # 只取训练集的前5%作为校准数据集
print("数据集加载完成")
#  对数据格式不清楚可以查看数据集的前几条数据
print(dataset[:3])


数据集加载完成
{'idx': [0, 1, 2], 'sentence': ['hide new secretions from the parental units ', 'contains no wit , only labored gags ', 'that loves its characters and communicates something rather beautiful about human nature '], 'label': [0, 0, 1]}


In [4]:

# 数据预处理函数
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)
# tokenizer 返回的是一个字典，包括 input_ids、attention_mask 等
encoded_dataset = dataset.map(preprocess_function, batched=True)
print("数据预处理完成")

数据预处理完成


# 使用Optimum配置量化参数(CPU量化)


## 库函数介绍
ORTModelForSequenceClassification 是一个专门用于序列分类任务的类，它封装了一个已经转换为 ONNX 格式的 Transformer 模型，并使用 ONNX Runtime 作为其推理引擎。

ORTQuantizer 用于创建量化器，它可以将 ONNX 模型转换为量化后的版本，以提高推理效率和减少模型大小。量化器支持多种量化方法和配置选项，可以根据具体需求进行调整。

AutoCalibrationDataLoader 是一个自动化的数据加载器，专门用于量化校准过程。它可以从给定的数据集中加载数据，并在推理期间观察激活值的分布，以便进行量化校准。该类简化了数据加载和预处理的过程，使得量化校准更加高效和便捷。

AutoQuantizationConfig 是一个自动化的量化配置类，用于定义量化过程中的各种参数和选项。它支持多种量化方法（如动态量化、静态量化等）和配置选项，可以根据具体需求进行调整。该类简化了量化配置的过程，使得量化操作更加高效和便捷。
## ONNX模型介绍
- ONNX（Open Neural Network Exchange）是一种开放的深度学习模型交换格式，旨在促进不同深度学习框架之间的互操作性。通过将模型导出为 ONNX 格式，可以在不同的运行时环境中使用该模型，而无需依赖于特定的深度学习框架。相比之下pytorch模型的运行需要各种依赖各种库。
- ONNX 模型文件通常以 `.onnx` 为扩展名，包含了模型的计算图、权重参数以及其他相关信息。ONNX 模型可以在支持 ONNX 格式的各种推理引擎中运行，如 ONNX Runtime、TensorRT 等。
- 使用 ONNX 模型的主要优势包括跨框架兼容性、优化的推理性能以及简化的部署流程。通过将模型转换为 ONNX 格式，可以更容易地在不同的平台和设备上部署深度学习模型。
- ONNX Runtime 是一个高性能的推理引擎，专门用于运行 ONNX 格式的深度学习模型。它支持多种硬件平台，包括 CPU、GPU 以及其他加速器，旨在提供快速且高效的模型推理能力。


In [5]:
from optimum.onnxruntime import ORTQuantizer, AutoQuantizationConfig, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoCalibrationConfig

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# 1. 加载并导出为ONNX模型
# 必须设置 export=True，因为原始模型是 PyTorch 格式，我们需要先将其转换为 ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_name, export=True)

# 2. 定义量化配置
# 使用 avx2 指令集优化，静态量化，逐通道量化
qconfig = AutoQuantizationConfig.avx2(is_static=True, per_channel=True)

# 3. 创建量化器
quantizer = ORTQuantizer.from_pretrained(ort_model)

# 4. 准备校准数据集 (关键修改：减少数据量)
# 崩溃原因：percentiles 方法需要记录所有激活值，数据量太大导致内存溢出 (OOM)
# 解决方法：只使用 100-200 条数据进行校准通常就足够了
columns_to_remove = [col for col in encoded_dataset.column_names if col not in tokenizer.model_input_names]
calibration_dataset = encoded_dataset.remove_columns(columns_to_remove).shuffle(seed=42).select(range(128))

# 5. 定义校准配置
# 使用 percentiles 方法，但现在数据量小了，不会崩溃
calibration_config = AutoCalibrationConfig.percentiles(calibration_dataset, percentile=99.995)

# 6. 执行校准
print("开始校准...")
ranges = quantizer.fit(
    dataset=calibration_dataset,
    calibration_config=calibration_config,
    operators_to_quantize=qconfig.operators_to_quantize,
)
print("校准完成")

# 7. 执行量化并保存
quantized_model_path = "./quantized_distilbert_sst2"
quantizer.quantize(
    save_dir=quantized_model_path,
    quantization_config=qconfig,
    calibration_tensors_range=ranges
)
print(f"量化完成！模型已保存至: {quantized_model_path}")

The model distilbert-base-uncased-finetuned-sst-2-english was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


开始校准...


/root/autodl-tmp/conda/envs/HiChunk_w/lib/python3.10/site-packages/numpy/lib/histograms.py:353: RuntimeWarning: overflow encountered in subtract
  return np.subtract(a, b, dtype=dt)


Finding optimal threshold for each tensor using 'percentile' algorithm ...
Number of tensors : 173
Number of histogram bins : 2048
Percentile : (0.0049999999999954525,99.995)
校准完成


量化完成！模型已保存至: ./quantized_distilbert_sst2


In [6]:
import os
import time
import numpy as np
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# 1. 加载量化后的模型
# 修改点3: 显式指定 file_name，解决 "Could not find any ONNX files" 的警告
quantized_model_path = "./quantized_distilbert_sst2"
quantized_model = ORTModelForSequenceClassification.from_pretrained(
    quantized_model_path, 
    file_name="model_quantized.onnx"
)

# 2. 定义辅助函数：获取目录大小
def get_dir_size(path):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

original_onnx_path = "./original_onnx_model"
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
ort_model = ORTModelForSequenceClassification.from_pretrained(model_name, export=False)
ort_model.save_pretrained(original_onnx_path)

# 3. 对比模型大小
original_size = get_dir_size(original_onnx_path) / (1024 * 1024) 
quantized_size = get_dir_size(quantized_model_path) / (1024 * 1024)

print(f"=== 模型大小对比 ===")
print(f"原始模型大小 (FP32 ONNX): {original_size:.2f} MB")
print(f"量化模型大小 (INT8 ONNX): {quantized_size:.2f} MB")
print(f"模型压缩率: {original_size / quantized_size:.2f}倍")

# 4. 对比推理速度
def measure_latency(model, tokenizer, text, n_loops=100):
    inputs = tokenizer(text, return_tensors="pt")
    for _ in range(10): model(**inputs) # Warmup
    
    start_time = time.time()
    for _ in range(n_loops):
        model(**inputs)
    end_time = time.time()
    return (end_time - start_time) / n_loops * 1000

text = "This movie is absolutely wonderful and I loved every moment of it."
original_latency = measure_latency(ort_model, tokenizer, text)
quantized_latency = measure_latency(quantized_model, tokenizer, text)

print(f"\n=== 推理速度对比 (CPU, Batch Size=1) ===")
print(f"原始模型平均耗时: {original_latency:.2f} ms")
print(f"量化模型平均耗时: {quantized_latency:.2f} ms")
print(f"速度提升: {original_latency / quantized_latency:.2f}倍")
print("注意：在 Batch Size=1 时，CPU 计算往往不是瓶颈，Python 调用开销占比较大，因此加速不明显。")

# 5. 对比输出精度
inputs = tokenizer(text, return_tensors="pt")
original_outputs = ort_model(**inputs).logits.detach().numpy()
quantized_outputs = quantized_model(**inputs).logits.detach().numpy()

print(f"\n=== 输出精度对比 ===")
print(f"原始模型 Logits: {original_outputs}")
print(f"量化模型 Logits: {quantized_outputs}")
mse = np.mean((original_outputs - quantized_outputs)**2)
print(f"均方误差 (MSE): {mse:.6f}")

=== 模型大小对比 ===
原始模型大小 (FP32 ONNX): 255.54 MB
量化模型大小 (INT8 ONNX): 65.39 MB
模型压缩率: 3.91倍

=== 推理速度对比 (CPU, Batch Size=1) ===
原始模型平均耗时: 21.11 ms
量化模型平均耗时: 21.93 ms
速度提升: 0.96倍
注意：在 Batch Size=1 时，CPU 计算往往不是瓶颈，Python 调用开销占比较大，因此加速不明显。

=== 输出精度对比 ===
原始模型 Logits: [[-4.3731527  4.7253337]]
量化模型 Logits: [[-4.2938247  4.6516433]]
均方误差 (MSE): 0.005862
